# Serving Runtime Remote Artifacts

In [1]:
import mlrun
import os

# Initialize the MLRun project object
project = mlrun.get_or_create_project('remote-artifacts',user_project=True,context='./')

# Required credentials :
# AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, GOOGLE_APPLICATION_CREDENTIALS, S3_BUCKET
AWS_ACCESS_KEY_ID = os.environ.get('AWS_ACCESS_KEY_ID', None)
AWS_SECRET_ACCESS_KEY = os.environ.get('AWS_SECRET_ACCESS_KEY', None)
GOOGLE_APPLICATION_CREDENTIALS = os.environ.get('GOOGLE_APPLICATION_CREDENTIALS', None)

# Setting as env variables
os.environ['AWS_ACCESS_KEY_ID'] = AWS_ACCESS_KEY_ID
os.environ['AWS_SECRET_ACCESS_KEY'] = AWS_SECRET_ACCESS_KEY
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = GOOGLE_APPLICATION_CREDENTIALS
assert AWS_ACCESS_KEY_ID != None and AWS_SECRET_ACCESS_KEY != None and GOOGLE_APPLICATION_CREDENTIALS != None, "please provide credentials"

secrets = {'AWS_ACCESS_KEY_ID': AWS_ACCESS_KEY_ID,
           'AWS_SECRET_ACCESS_KEY':AWS_SECRET_ACCESS_KEY,
           'GOOGLE_APPLICATION_CREDENTIALS': GOOGLE_APPLICATION_CREDENTIALS}

project.set_secrets(secrets=secrets, provider='kubernetes')

S3_BUCKET = os.environ.get('S3_BUCKET', 'testbucket-igz-temp')

assert project.artifact_path != None, 'please run 01_mlrun_job.ipynb'

> 2023-03-06 07:39:46,471 [info] loaded project remote-artifacts from MLRun DB


In [2]:
# Setting dummy serving function
project.set_function(mlrun.new_function(name='serving', kind='serving',image='mlrun/mlrun', command = []))
project.get_function('serving').spec.build.commands = ['pip install xgboost']
project.get_function('serving').set_envs(secrets)
# adding our remote source model
model_artifact = project.get_artifact('log-transactions-log-transactions_clf_model')
model_path = model_artifact.get_target_path() + model_artifact.model_file

project.get_function('serving').add_model(key = 'my_model',
                                          class_name = "mlrun.frameworks.xgboost.XGBoostModelServer",
                                          model_path = model_path)

# mounting
project.get_function('serving').apply(mlrun.auto_mount())
project.get_function('serving').apply(mlrun.platforms.mount_s3())

> 2023-03-06 07:39:46,557 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe


In [3]:
server = project.get_function('serving').to_mock_server()

> 2023-03-06 07:39:46,569 [warning] run command, file or code were not specified
> 2023-03-06 07:39:58,999 [info] model my_model was loaded
> 2023-03-06 07:39:59,007 [info] Loaded ['my_model']


In [4]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X,y = load_iris(return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

server.test(body={'inputs': X_test.tolist()[:3]})

{'id': 'c69b7ad680264048be4629f97ca75e94',
 'model_name': 'my_model',
 'outputs': [2, 1, 0]}

In [5]:
project.deploy_function('serving')

> 2023-03-06 07:39:59,065 [info] Starting remote function deploy
2023-03-06 07:39:59  (info) Deploying function
2023-03-06 07:39:59  (info) Building
2023-03-06 07:39:59  (info) Staging files and preparing base images
2023-03-06 07:39:59  (info) Building processor image
2023-03-06 07:40:59  (info) Build complete
2023-03-06 07:41:15  (info) Function deploy complete
> 2023-03-06 07:41:20,280 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-remote-artifacts-normal-user-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['remote-artifacts-normal-user-serving-remote-artifacts-1tkokejo.default-tenant.app.vmdev92.lab.iguazeng.com/']}


DeployStatus(state=ready, outputs={'endpoint': 'http://remote-artifacts-normal-user-serving-remote-artifacts-1tkokejo.default-tenant.app.vmdev92.lab.iguazeng.com/', 'name': 'remote-artifacts-normal-user-serving'})

In [6]:
import time
time.sleep(5)
project.get_function('serving').invoke('/v2/models/my_model/infer', body={'inputs': X_test.tolist()[:3]})

> 2023-03-06 07:41:25,376 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-remote-artifacts-normal-user-serving.default-tenant.svc.cluster.local:8080/v2/models/my_model/infer'}


{'id': 'dd19d122-4768-4e4f-944f-fd64e060d66f',
 'model_name': 'my_model',
 'outputs': [2, 1, 0]}

# Pipeline remote access

In [7]:
%%writefile workflow.py
import mlrun
from kfp import dsl
import os

@dsl.pipeline(name="Mask Detection Pipeline")

def kfpipeline():
    # Get our project object:
    project = mlrun.get_current_project()
    
    project.get_function('log_transactions').apply(mlrun.platforms.set_env_variables({'GOOGLE_APPLICATION_CREDENTIALS':
                                                                                      os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')}))
    project.get_function('log_transactions').apply(mlrun.platforms.mount_s3())
    project.get_function('log_transactions').apply(mlrun.auto_mount())
    run = project.run_function('log_transactions')
    
    model_artifact = project.get_artifact('log-transactions-log-transactions_clf_model')
    model_path = model_artifact.get_target_path() + model_artifact.model_file
    
    project.get_function('serving').add_model(key = 'my_model',
                                          class_name = "mlrun.frameworks.xgboost.XGBoostModelServer",
                                          model_path = model_path)
    
    project.deploy_function('serving').after(run)
    return

Writing workflow.py


In [8]:
# Register the workflow file:
workflow_name = "iris_workflow"
project.set_workflow(workflow_name, "workflow.py")

# Save the project:
project.save()

In [9]:
# Running with S3
project.run(name=workflow_name,watch=True, local=True, artifact_path=project.artifact_path)
project.save()

> 2023-03-06 07:41:56,176 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2023-03-06 07:41:56,182 [info] starting run log-transactions-log-transactions uid=a3e7aca978e543d4bd67ab7b6bb93617 DB=http://mlrun-api:8080
> 2023-03-06 07:42:05,090 [info] encoded dataframe shape : (150, 4)
> 2023-03-06 07:42:06,114 [info] logged model : <class 'xgboost.sklearn.XGBClassifier'>
> 2023-03-06 07:42:17,861 [info] logged dataset shape (150, 5)


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
remote-artifacts-normal-user,...6bb93617,0,Mar 06 07:41:56,completed,log-transactions-log-transactions,workflow=46e1cfbe5d0542728ab0d79cc93b66bfv3io_user=normal-userkind=owner=normal-userhost=jupyter-v5afxg4hcf-at3sd-77c89bbcd5-svv9f,,,,encoded_iris-s3clf_modelmy_projectiris_dataset-s3


> 2023-03-06 07:42:18,171 [info] run executed, status=completed
> 2023-03-06 07:42:18,210 [info] Starting remote function deploy
2023-03-06 07:42:18  (info) Deploying function
2023-03-06 07:42:18  (info) Building
2023-03-06 07:42:18  (info) Staging files and preparing base images
2023-03-06 07:42:18  (info) Building processor image
2023-03-06 07:43:08  (info) Build complete
2023-03-06 07:43:22  (info) Function deploy complete
> 2023-03-06 07:43:29,210 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-remote-artifacts-normal-user-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['remote-artifacts-normal-user-serving-remote-artifacts-jg08pn78.default-tenant.app.vmdev92.lab.iguazeng.com/']}


uid,start,state,name,parameters,results
...6bb93617,Mar 06 07:41:56,completed,log-transactions-log-transactions,,


> 2023-03-06 07:43:29,284 [info] started run workflow remote-artifacts-normal-user-iris_workflow with run id = '46e1cfbe5d0542728ab0d79cc93b66bf' by local engine


In [10]:
# Running with GCS
project.run(name=workflow_name,watch=True, local=True, artifact_path='gs' + project.artifact_path[2:])

> 2023-03-06 07:43:29,540 [warning] it is recommended to use k8s secret (specify secret_name), specifying the aws_access_key/aws_secret_key directly is unsafe
> 2023-03-06 07:43:29,550 [info] starting run log-transactions-log-transactions uid=46d99172631d4ed2b8b6e76afad53920 DB=http://mlrun-api:8080
> 2023-03-06 07:43:30,786 [info] encoded dataframe shape : (150, 4)
> 2023-03-06 07:43:32,381 [info] logged model : <class 'xgboost.sklearn.XGBClassifier'>
> 2023-03-06 07:43:34,328 [info] logged dataset shape (150, 5)


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
remote-artifacts-normal-user,...fad53920,0,Mar 06 07:43:29,completed,log-transactions-log-transactions,workflow=db73332bb0a04d978c0139aade1cbd56v3io_user=normal-userkind=owner=normal-userhost=jupyter-v5afxg4hcf-at3sd-77c89bbcd5-svv9f,,,,encoded_iris-gsclf_modelmy_projectiris_dataset-gs


> 2023-03-06 07:43:34,516 [info] run executed, status=completed
> 2023-03-06 07:43:34,548 [info] Starting remote function deploy
2023-03-06 07:43:34  (info) Deploying function
2023-03-06 07:43:34  (info) Building
2023-03-06 07:43:34  (info) Staging files and preparing base images
2023-03-06 07:43:34  (info) Building processor image
2023-03-06 07:44:21  (info) Build complete
2023-03-06 07:44:33  (info) Function deploy complete
> 2023-03-06 07:44:35,293 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-remote-artifacts-normal-user-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['remote-artifacts-normal-user-serving-remote-artifacts-yoy50axc.default-tenant.app.vmdev92.lab.iguazeng.com/']}


uid,start,state,name,parameters,results
...fad53920,Mar 06 07:43:29,completed,log-transactions-log-transactions,,


> 2023-03-06 07:44:35,370 [info] started run workflow remote-artifacts-normal-user-iris_workflow with run id = 'db73332bb0a04d978c0139aade1cbd56' by local engine


db73332bb0a04d978c0139aade1cbd56